In [4]:
!pip -q install redis redis-cli

In [ ]:
import redis
r = redis.Redis(host="127.0.0.1", port=6379, db=0)

````markdown
## Redis Sorted Sets Explained - [Link to Video](https://youtu.be/XqSK-4oEoAc)

### Overview: Real-Time Leaderboard with Sorted Sets
A **Redis Sorted Set** is an ordered collection of unique members, each associated with a numeric **score** that determines ordering (ties break lexicographically) ([docs](https://redis.io/docs/latest/develop/data-types/sorted-sets/)). This fits **Mages & Minotaurs** leaderboards where experience points (XP) change frequently and must be queried with low latency. Typical uses include **leaderboards**, **priority queues** (use the score as priority), and **secondary indexing** ([overview](https://redis.io/docs/latest/develop/data-types/sorted-sets/), [priority queue pattern](https://redis.io/glossary/redis-queue/), [secondary index guide](https://redis.io/docs/latest/develop/clients/patterns/indexes/)).

> **Sidenote**  
>  
> [Link: Redis Sorted Sets](https://redis.io/docs/latest/develop/data-types/sorted-sets/)  
>  
> **Concept**: `Sorted Set ordering` → Members are unique strings ordered by a floating-point score; ties are resolved lexicographically.  
>  
> **Context**: Used to maintain a continuously ordered leaderboard as players earn XP.  
>  
> **Example**: Player IDs are members; XP is the score.  
>  
> **Implication**:  
>  
> Constantly updated scores maintain rank order without re-sorting; rank queries and top-N retrievals stay efficient.

---

### Create the Leaderboard and Insert Players
Use `ZADD` to create `leaders:exp` and add members with initial scores (e.g., new players start at 0 XP). `ZADD` adds or updates members and returns the number of **new** elements added ([ZADD](https://redis.io/docs/latest/commands/zadd/)).

```python
# Initialize the leaderboard; each player starts at 0 XP
r.execute_command("ZADD", "leaders:exp", 0, "42")
print("1 new member added")  # verification: ZADD returns number of new members

# (Optional) Add more players with initial score 0
# Example loop (conceptual): r.execute_command("ZADD", "leaders:exp", 0, "<player_id>")
print("additional members added")  # verification placeholder

# Verify total members
r.execute_command("ZCARD", "leaders:exp")
print("total count shown")  # e.g., 12 after adding 11 more
````

> **Sidenote**
>
> [Link: ZADD](https://redis.io/docs/latest/commands/zadd/)
>
> **Command**: `ZADD` → Add one or more members with scores to a sorted set; update score if member exists.
>
> **Pattern**: `ZADD key [options] score member [score member ...]`
>
> **Example**: `ZADD leaders:exp 0 42`
>
> **Result**:
>
> Integer (count of **new** members added). Options like `NX|XX|GT|LT|CH|INCR` refine behavior.

---

### Increment (or Decrement) a Player’s Score in Real Time

Use `ZINCRBY` to add XP to a player as events occur; negative increments decrement XP. Returns the **new score** as a bulk string in RESP2 or double in RESP3 ([ZINCRBY](https://redis.io/docs/latest/commands/zincrby/)).

```python
# Player 42 defeats a monster and earns 300 XP
r.execute_command("ZINCRBY", "leaders:exp", 300, "42")
print("300")  # verification: new score returned as a string (RESP2)

# (Optional) Decrement by using a negative increment
# r.execute_command("ZINCRBY", "leaders:exp", -50, "42")
# print("new score after penalty")  # verification

# Verify via ZSCORE
r.execute_command("ZSCORE", "leaders:exp", "42")
print("300")  # verification: should match current score
```

> **Sidenote**
>
> [Link: ZINCRBY](https://redis.io/docs/latest/commands/zincrby/)
>
> **Command**: `ZINCRBY` → Increase/decrease a member’s score by an increment (creates member if absent).
>
> **Pattern**: `ZINCRBY key increment member`
>
> **Example**: `ZINCRBY leaders:exp 300 42`
>
> **Result**:
>
> New score of the member (bulk string/double). Accepts negative increments.

---

### Display the Top 10 Players (Highest to Lowest)

To list top N players by score descending, the transcript uses `ZREVRANGE ... WITHSCORES`. Note: `ZREVRANGE` is **deprecated** in favor of `ZRANGE ... REV` since Redis 6.2; both are shown for clarity ([ZREVRANGE](https://redis.io/docs/latest/commands/zrevrange/), [ZRANGE](https://redis.io/docs/latest/commands/zrange/)).

```python
# Get top 10 players (highest scores first) with their scores
r.execute_command("ZREVRANGE", "leaders:exp", 0, 9, "WITHSCORES")
print("[(player_id, score), ...] 10 items")  # verification: top 10 returned

# Preferred modern equivalent using ZRANGE REV:
# r.execute_command("ZRANGE", "leaders:exp", 0, 9, "REV", "WITHSCORES")
# print("[(player_id, score), ...] 10 items")  # verification
```

> **Sidenote**
>
> [Link: ZREVRANGE](https://redis.io/docs/latest/commands/zrevrange/) • [Link: ZRANGE (with REV)](https://redis.io/docs/latest/commands/zrange/)
>
> **Command**: `ZREVRANGE` → Return members by **descending** score; `WITHSCORES` includes scores.
>
> **Pattern**: `ZREVRANGE key start stop [WITHSCORES]`
>
> **Example**: `ZREVRANGE leaders:exp 0 9 WITHSCORES`
>
> **Result**:
>
> Array of members (optionally interleaved with scores). Use `ZRANGE ... REV` for modern, unified ranges.

---

### Get a Player’s Rank (Position in the Leaderboard)

**Rank** is the zero-based index of a member within the sorted set under a given ordering.

* `ZRANK` uses **ascending** order (lowest score → rank 0).
* `ZREVRANK` uses **descending** order (highest score → rank 0).
  Ranks are **0-based** ([ZRANK](https://redis.io/docs/latest/commands/zrank/), [ZREVRANK](https://redis.io/docs/latest/commands/zrevrank/)).

```python
# Ascending rank: 0 = lowest score
r.execute_command("ZRANK", "leaders:exp", "42")
print("11")  # verification example from transcript: 12 members → 42 is highest → rank 11 ascending

# Descending rank: 0 = highest score
r.execute_command("ZREVRANK", "leaders:exp", "42")
print("0")  # verification: player 42 is current leader
```

> **Sidenote**
>
> [Link: ZRANK](https://redis.io/docs/latest/commands/zrank/) • [Link: ZREVRANK](https://redis.io/docs/latest/commands/zrevrank/)
>
> **Command**: `ZRANK` / `ZREVRANK` → Return zero-based position of a member (ascending vs. descending).
>
> **Pattern**: `ZRANK key member [WITHSCORE]` • `ZREVRANK key member [WITHSCORE]`
>
> **Example**: `ZREVRANK leaders:exp 42` → `0`
>
> **Result**:
>
> Integer rank (or `nil` if absent). `WITHSCORE` (Redis ≥7.2) can include the score.

---

### Get a Player’s Current Score

Use `ZSCORE` to fetch the exact XP for a given player ID ([ZSCORE](https://redis.io/docs/latest/commands/zscore/)).

```python
# Fetch the exact XP for player 42
r.execute_command("ZSCORE", "leaders:exp", "42")
print("300")  # verification: score returned as string in RESP2; double in RESP3
```

> **Sidenote**
>
> [Link: ZSCORE](https://redis.io/docs/latest/commands/zscore/)
>
> **Command**: `ZSCORE` → Get the score for a member.
>
> **Pattern**: `ZSCORE key member`
>
> **Example**: `ZSCORE leaders:exp 42`
>
> **Result**:
>
> Bulk string/double representing the member’s score; `nil` if member/key doesn’t exist.

---

### Concept Recap (Technical)

* Sorted Sets store **unique** members ordered by **score**; ties sort **lexicographically** ([sorted sets](https://redis.io/docs/latest/develop/data-types/sorted-sets/), [ZADD tie-breaks](https://redis.io/docs/latest/commands/zadd/#elements-with-the-same-score)).
* You can traverse in **ascending** or **descending** order; `ZRANGE` supports `REV`, `BYSCORE`, `BYLEX` ([ZRANGE](https://redis.io/docs/latest/commands/zrange/)).
* Scores are mutable via `ZADD` (update) or `ZINCRBY` (increment/decrement), keeping rank order current ([sorted sets: updates & leaderboards](https://redis.io/docs/latest/develop/data-types/sorted-sets/), [ZINCRBY](https://redis.io/docs/latest/commands/zincrby/)).
* Common applications: **leaderboards**, **priority queues** (score = priority), and **secondary indexes** ([overview](https://redis.io/docs/latest/develop/data-types/sorted-sets/), [priority queue pattern](https://redis.io/glossary/redis-queue/), [secondary index guide](https://redis.io/docs/latest/develop/clients/patterns/indexes/)).


### Original Transcript

Video title: Redis Sorted Sets Explained Video URL: https://youtu.be/XqSK-4oEoAc Video language: English (United States) -------------------------------- How do we create a real-time leaderboard for Mages & Minotaurs? Redis' solution to this is a Sorted Set, and it's a tool every developer should know. So a Redis Sorted Set is an ordered collection of unique members. These Set members are ordered by their associated score. Whenever you add to a Sorted Set, you're specifying a member and a score. Sorted Sets keep everything sorted from the get-go. Sorted Sets are a good choice for priority queues, low-latency leaderboards, and secondary indexing in general. To see how we can use Sorted Sets, let's imagine we're building an online role-playing game called Mages & Minotaurs. In a massive game like this, experience points in particular will be changing all the time. It'll be an excellent example for a player leaderboard using a Sorted Set. A Redis Sorted Set provides the responsive in-memory access that a real-time game demands. Once we're efficiently storing players' experience points in a Sorted Set, there's three things we're going to want to do. First, we'll want to display the top players. Second, we'll want to be able to display the rank of any given player. And finally, we'll also want to see any given player's score in the game. Let's look at the Sorted Set commands that enable these features. Let's start with creating a leaderboard, inserting our first player, and incrementing their score value. To add a score member pair to a Redis Sorted Set, use the ZADD command, followed by the key, the score, and the member. The key identifying our Sorted Set will be leaders:exp. Each player starts with no experience, so their initial score will be 0. For the member, we'll store the unique player ID. Let's use the ID of 42. ZADD returns the number of new members stored in the Sorted Set -- in this case, 1. I'll add 11 more players as part of this example. Now that we've added the players to a Sorted Set, let's start everything in motion. As the game progresses, the players' experience points will increase from finishing quests, battling enemies, and discovering items. We'll add the experience points every player accumulates in real time to our Sorted Set. To increment a member's score by a given number, use the ZINCRBY command, which takes three arguments -- the key, the increment value, and the member whose score you want to increase. If the player with an ID of 42 defeats a monster and earns 300 experience points, the command will be ZINCRBY leaders:exp 300 42. ZINCRBY returns the new score, 300, as a string. By the way, we can also decrement scores by using a negative integer. I won't punish this player, but know that this is a common practice. Now that we have our Sorted Set populated with members and their scores, we can access leaders:exp as a leaderboard and glean some interesting information. What if we want to get a list of the top 10 players in the game? There are two commands that iterate over a Sorted Set in order of score and return the resulting members -- ZRANGE and ZREVRANGE. ZRANGE traverses a range of members from the lowest score to the highest score. ZREVRANGE does the opposite. It traverses a range from highest to lowest score. So for this use case, it's easiest to use a ZREVRANGE command. The full command we'll use to view the top 10 players in our leaderboard with their scores is ZREVRANGE leaders:exp 0 9 WITHSCORES. The 0 and 9 refer to the starting and ending indexes of our range capture. ZREVRANGE's WITHSCORES option includes the associated scores in the output. We now have a list of the top 10 player IDs of the highest experience points in ascending order. What if you wanted to display the rank of only a single member, say, on a player dashboard? Rank is a relative position by score of the member in the Sorted Set. A rank of 0 refers to the members with the lowest associated score. A rank of 1 represents the member with the second smallest associated score. We use the ZRANK command to get a member's rank from a Sorted Set. If you were to run the ZRANK command on the player ID 42, the number 11 would be returned. This means in a Sorted Set of length 12, the player 42 is in the highest rank, having the highest associated score. This rank value isn't exactly intuitive, so we can use the ZREVRANK command. ZREVRANK gives us a ranking with a reverse sorting from highest to lowest. ZREVRANK leaders:exp 42 returns the number 0, which would mean that member has the highest score. For our particular Sorted Set, this means that player 42 is our experience points leader. Finally, how would we go about displaying the experience points of a member on their dashboard or, say, that of their enemy or guild member? We'll use the command ZSCORE, which takes a key and a single member as arguments. When we enter ZSCORE leaders:exp 42, the score, 300, is returned as a string. OK, let's do a quick review. Sorted Sets are an ordered collection of unique values. Each member has an associated score which is used to order the members of the Sorted Set from lowest to highest score values. The Sorted Set can be accessed in ascending or descending order. The scores in Sorted Sets can be incremented or decremented, which will change the order of the members. To learn more about Redis Sorted Sets, check out our free online course, "Introduction to Redis Data Structures". It's part of Redis University, our online learning platform for all things Redis. Thanks for joining me in this quick dive into Redis Sorted Sets. Happy learning, and see you again soon.